In [1]:
!pip install deepface retina-face

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00


In [2]:
# Emotion Map
# DeepFace has angry, fear, neutral, sad, disgust, happy, and surprise
# Dr. Fatma M Tal's dataset includes natural, anger, fear, joy, sadness, surprise


from deepface import DeepFace
from tqdm import tqdm
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix

DF2LBL = {
    "angry": "anger",
    "disgust": "anger",   # merged
    "fear": "fear",
    "happy": "joy",
    "sad": "sadness",
    "surprise": "surprise",
    "neutral": "neutral",
}

CLASSES = ["neutral","anger","fear","joy","sadness","surprise"]

2025-08-23 17:19:35.030811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755969575.229095      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755969575.298018      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


25-08-23 17:19:47 - Directory /root/.deepface has been created
25-08-23 17:19:47 - Directory /root/.deepface/weights has been created


In [3]:
def analyze_image(path, detector_backend="retinaface", enforce_detection=False):
    """
    Returns (pred_label, mapped_probs_dict) or (None, None) on failure.
    """
    try:
        res = DeepFace.analyze(
            img_path=path,
            actions=["emotion"],
            detector_backend=detector_backend,
            enforce_detection=enforce_detection
        )
        if isinstance(res, list):
            res = res[0]
        probs = res.get("emotion", {})
        if not probs:
            return None, None

        probs = {k.lower(): float(v) for k,v in probs.items()}
        mapped = {}
        for k, v in probs.items():
            if k in DF2LBL:
                mapped.setdefault(DF2LBL[k], 0.0)
                mapped[DF2LBL[k]] += v
        for c in CLASSES:
            mapped.setdefault(c, 0.0)
        pred = max(mapped.items(), key=lambda kv: kv[1])[0]
        return pred, mapped
    except Exception:
        return None, None

In [4]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("/kaggle/input/autistic-children-emotions-dr-fatma-m-talaat/Autistic Children Emotions - Dr. Fatma M. Talaat/Test")

FOLDER2LABEL = {
    "Natural": "neutral",
    "neutral": "neutral",
    "anger": "anger",
    "fear": "fear",
    "joy": "joy",
    "sadness": "sadness",
    "surprise": "surprise",
}

rows = []
for sub in DATA_DIR.iterdir():
    if not sub.is_dir(): 
        continue
    lbl = FOLDER2LABEL.get(sub.name, None)
    if lbl is None:
        continue
    for p in sub.rglob("*"):
        if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp"}:
            rows.append({"image_path": str(p), "label": lbl})


df = pd.DataFrame(rows)
print(df.shape)
df.head()

(75, 2)


,image_path,label
0,/kaggle/input/autistic-children-emotions-dr-fa...,joy
1,/kaggle/input/autistic-children-emotions-dr-fa...,joy
2,/kaggle/input/autistic-children-emotions-dr-fa...,joy
3,/kaggle/input/autistic-children-emotions-dr-fa...,joy
4,/kaggle/input/autistic-children-emotions-dr-fa...,joy


In [5]:
y_true, y_pred = [], []
pred_rows = []

for _, r in tqdm(df.iterrows(), total=len(df)):
    path = r["image_path"]
    true_lbl = r["label"]
    pred, probs = analyze_image(path, detector_backend="retinaface", enforce_detection=False)
    if pred is None:
        continue
    
    y_true.append(true_lbl)
    y_pred.append(pred)

    row = {"image_path": path, "label": true_lbl, "pred": pred}
    for c in CLASSES:
        row[f"prob_{c}"] = probs.get(c, 0.0)
    pred_rows.append(row)

pred_df = pd.DataFrame(pred_rows)
print(pred_df.shape)
pred_df.head()

  0%|          | 0/75 [00:00<?, ?it/s]I0000 00:00:1755969588.404001      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


25-08-23 17:19:51 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
  9%|▉         | 11.0M/119M [00:00<00:01, 88.5MB/s]
 23%|██▎       | 27.3M/119M [00:00<00:00, 128MB/s] 
 36%|███▌      | 42.5M/119M [00:00<00:00, 113MB/s]
 48%|████▊     | 57.1M/119M [00:00<00:00, 125MB/s]
 61%|██████    | 71.8M/119M [00:00<00:00, 131MB/s]
 72%|███████▏  | 85.5M/119M [00:00<00:00, 118MB/s]
 84%|████████▍ | 100M/119M [00:00<00:00, 126MB/s] 
100%|██████████| 119M/119M [00:00<00:00, 122MB/s]
I0000 00:00:1755969595.369538     109 cuda_dnn.cc:529] Loaded cuDNN version 90300
Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5


25-08-23 17:19:57 - 🔗 facial_expression_model_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5 to /root/.deepface/weights/facial_expression_model_weights.h5...



100%|██████████| 5.98M/5.98M [00:00<00:00, 105MB/s]
100%|██████████| 75/75 [01:06<00:00,  1.13it/s]

(75, 9)


,image_path,label,pred,prob_neutral,prob_anger,prob_fear,prob_joy,prob_sadness,prob_surprise
0,/kaggle/input/autistic-children-emotions-dr-fa...,joy,joy,2.565525e-10,2.751258e-13,1.079476e-15,100.000000,8.957598e-11,7.562898e-13
1,/kaggle/input/autistic-children-emotions-dr-fa...,joy,neutral,8.223849e+01,1.411838e+00,1.608385e-03,0.299032,1.604902e+01,1.123623e-05
2,/kaggle/input/autistic-children-emotions-dr-fa...,joy,joy,3.486378e-02,5.563329e-03,3.422963e-07,99.957341,2.212790e-03,1.663669e-05
3,/kaggle/input/autistic-children-emotions-dr-fa...,joy,fear,4.913207e+00,2.323841e-02,6.857584e+01,2.244788,2.424293e+01,6.598571e-07
4,/kaggle/input/autistic-children-emotions-dr-fa...,joy,joy,3.260836e+00,4.492074e-07,2.823100e-10,96.738607,1.598093e-05,5.430802e-04


In [6]:
acc = accuracy_score(y_true, y_pred)
bacc = balanced_accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro")

print(f"Accuracy: {acc:.4f}")
print(f"Balanced Accuracy: {bacc:.4f}")
print(f"Macro-F1: {macro_f1:.4f}\n")

print("Per-class metrics:")
print(classification_report(y_true, y_pred, labels=CLASSES, digits=4, zero_division=0))

cm = confusion_matrix(y_true, y_pred, labels=CLASSES)
pd.DataFrame(cm, index=CLASSES, columns=CLASSES)

Accuracy: 0.5867
Balanced Accuracy: 0.2817
Macro-F1: 0.2604

Per-class metrics:
              precision    recall  f1-score   support

     neutral     0.1429    0.2857    0.1905         7
       anger     0.2500    0.3333    0.2857         3
        fear     0.0000    0.0000    0.0000         3
         joy     0.8478    0.9286    0.8864        42
     sadness     0.3333    0.1429    0.2000        14
    surprise     0.0000    0.0000    0.0000         6

    accuracy                         0.5867        75
   macro avg     0.2623    0.2817    0.2604        75
weighted avg     0.5603    0.5867    0.5629        75



,neutral,anger,fear,joy,sadness,surprise
neutral,2,0,2,1,2,0
anger,1,1,0,1,0,0
fear,1,0,0,1,1,0
joy,1,0,1,39,1,0
sadness,7,1,1,3,2,0
surprise,2,2,1,1,0,0


In [7]:
import pandas as pd
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix

pred_df.head()


label_to_zone = {
    "joy": "Positive",
    "neutral": "Positive",    
    "Natural": "Positive",   
    "anger": "NegativeActive",
    "fear": "NegativeActive",
    "surprise": "NegativeActive",
    "sadness": "NegativePassive",
}

gt_zones = pred_df["label"].map(lambda x: label_to_zone.get(str(x).lower(), "UNKNOWN"))
pred_zones = pred_df["pred"].map(lambda x: label_to_zone.get(str(x).lower(), "UNKNOWN"))

mask = (gt_zones != "UNKNOWN") & (pred_zones != "UNKNOWN")
gt_z = gt_zones[mask]
pd_z = pred_zones[mask]

zone_classes = ["Positive", "NegativeActive", "NegativePassive"]

acc = accuracy_score(gt_z, pd_z)
bacc = balanced_accuracy_score(gt_z, pd_z)
macro_f1 = f1_score(gt_z, pd_z, average="macro")

print(f"Zone Accuracy: {acc:.4f}")
print(f"Zone Balanced Accuracy: {bacc:.4f}")
print(f"Zone Macro-F1: {macro_f1:.4f}\n")

print("Per-zone metrics:")
print(classification_report(gt_z, pd_z, labels=zone_classes, digits=4, zero_division=0))

cm = confusion_matrix(gt_z, pd_z, labels=zone_classes)
pd.DataFrame(cm, index=[f"true_{c}" for c in zone_classes], columns=[f"pred_{c}" for c in zone_classes])


Zone Accuracy: 0.6533
Zone Balanced Accuracy: 0.4512
Zone Macro-F1: 0.4566

Per-zone metrics:
                 precision    recall  f1-score   support

       Positive     0.7167    0.8776    0.7890        49
 NegativeActive     0.4444    0.3333    0.3810        12
NegativePassive     0.3333    0.1429    0.2000        14

       accuracy                         0.6533        75
      macro avg     0.4981    0.4512    0.4566        75
   weighted avg     0.6016    0.6533    0.6138        75



,pred_Positive,pred_NegativeActive,pred_NegativePassive
true_Positive,43,3,3
true_NegativeActive,7,4,1
true_NegativePassive,10,2,2
